In [ ]:
## charger env_ISMIP6 ou env_MIP

In [ ]:
# module import
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.tri as tri
import cartopy.crs as ccrs
import pyproj
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.path as mpath
import seaborn as sns
import datetime
import cftime
from matplotlib import gridspec
import cartopy.feature as cft
import pandas as pd
import string
import cmocean as cm
import math
import proplot as pplt
import xesmf as xe
import netCDF4 as nc
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.image as mpimg

### data for calculation of SLE
* Methods used is similar to Goelzer et al. (2020)
* Area of ocean is from Gregory et al. (2019) (3.625e14 m2)
* rho_ice = 917 kg.m-3 (as Elmer ISMIP6 configuration)
* rho_ocean = 1028 kg.m-3 (as Elmer ISMIP6 configuration)

In [ ]:
# constant
A_ocean = 3.625e14 # m2
rho_ice = 917 # kg.m-3
rho_ocean = 1028 # kg.m-3
rho_water = 1000 # kg.m-3

# conversion kg.m-2.s-1 to kg.yr-1
fac = 4*1e3 * 4*1e3 * 86400 * 365

## UKESM

### atmosphere

In [ ]:
# Open data 
netCDF_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/MAR-UKESM1-0-LL-r10i1p1f2_asmb_2015-2100_ssp245_regrid_04000m_FROM_UKESM1-0-LL-r1i1p1f2-ssp245.nc'
ATMOU = xr.open_mfdataset(netCDF_file, decode_times=False)

netCDF_filemask = '/Users/jcaillet/Documents/Elmer/DATA/BASINS/Mask_SubRegions_IMBIE.nc'
maskS = xr.open_mfdataset(netCDF_filemask)

netCDF_filemask = '/Users/jcaillet/Documents/Elmer/DATA/BASINS/Mask_Regions_IMBIE.nc'
maskR = xr.open_mfdataset(netCDF_filemask)

netCDF_ini = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/RCM_ice_regrid_04000m.nc2'
ini = xr.open_mfdataset(netCDF_ini)

# available members
membu = np.array([1,2,4,8])
# bassins
subbas = np.arange(1,19,1)
bas = np.arange(1,4,1)
basf = np.arange(0,4,1)

# interpolation of mask on the same grid as MAR data
ATMOU['x']=ini['x']
ATMOU['y']=ini['y']
maskS_interp = maskS.interp_like(ATMOU)
maskR_interp = maskR.interp_like(ATMOU)

#initialisation
t = np.arange(2015,2101,1)
FINAL_FILEAU = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

#boucle sur membres
for j in membu :
    data_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/MAR-UKESM1-0-LL-r{j}i1p1f*_asmb_2015-2100_ssp245_regrid_04000m_FROM_UKESM1-0-LL-r1i1p1f2-ssp245.nc'
    RES = xr.open_mfdataset(data_file, decode_times=False)
    RES['time']=FINAL_FILEAU['time']
    # boucle sur bassins
    for i in subbas:
        FINAL_FILEAU[f'SLEA_SB{i}_{j}'] = (RES['asmb'] * fac * ini['af2'] * ini['GROUND']).where(maskS['SubBasins_extrap']==i).sum(['x','y'])
        FINAL_FILEAU[f'SLEA_SB{i}_{j}'] = -(np.cumsum(FINAL_FILEAU[f'SLEA_SB{i}_{j}']) * 1 / (A_ocean * rho_water))
    for i in bas:
        FINAL_FILEAU[f'SLEA_B{i}_{j}'] = (RES['asmb'] * fac * ini['af2'] * ini['GROUND']).where(maskR['Basins_extrap']==i).sum(['x','y'])
        FINAL_FILEAU[f'SLEA_B{i}_{j}'] = -(np.cumsum(FINAL_FILEAU[f'SLEA_B{i}_{j}']) * 1 / (A_ocean * rho_water))       
    for i in np.array([0]):
        #projection
        FINAL_FILEAU[f'SLEA_B{i}_{j}'] = (RES['asmb'] * fac * ini['af2'] * ini['GROUND']).sum(['x','y'])
        FINAL_FILEAU[f'SLEA_B{i}_{j}'] = -(np.cumsum(FINAL_FILEAU[f'SLEA_B{i}_{j}']) * 1 / (A_ocean * rho_water))
# SLE en m !

In [ ]:
FINAL_FILEAU100 = xr.Dataset(data_vars=dict(memb=(["member"], membu)), coords=dict(member=membu))
t = np.arange(2015,2101,1)
MIN_MAXAU = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

for i in subbas:
    print(i)
    d = {'M1': np.array(FINAL_FILEAU[f'SLEA_SB{i}_1']), 'M2': np.array(FINAL_FILEAU[f'SLEA_SB{i}_2']), 'M3': np.array(FINAL_FILEAU[f'SLEA_SB{i}_3']), 'M4': np.array(FINAL_FILEAU[f'SLEA_SB{i}_4']), 'M5': np.array(FINAL_FILEAU[f'SLEA_SB{i}_5']), 'M6': np.array(FINAL_FILEAU[f'SLEA_SB{i}_6']), 'M7': np.array(FINAL_FILEAU[f'SLEA_SB{i}_7']), 'M8': np.array(FINAL_FILEAU[f'SLEA_SB{i}_8']), 'M9': np.array(FINAL_FILEAU[f'SLEA_SB{i}_9']), 'M10': np.array(FINAL_FILEAU[f'SLEA_SB{i}_10']), 'M11': np.array(FINAL_FILEAU[f'SLEA_SB{i}_11']), 'M12': np.array(FINAL_FILEAU[f'SLEA_SB{i}_12']), 'M13': np.array(FINAL_FILEAU[f'SLEA_SB{i}_13']), 'M16': np.array(FINAL_FILEAU[f'SLEA_SB{i}_16']), 'M17': np.array(FINAL_FILEAU[f'SLEA_SB{i}_17']), 'M18': np.array(FINAL_FILEAU[f'SLEA_SB{i}_18']), 'M19': np.array(FINAL_FILEAU[f'SLEA_SB{i}_19'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEAU100[f'SLEA_SB{i}']= FINAL_FILEAU100['memb'] - FINAL_FILEAU100['memb'] + df2100
    MIN_MAXAU[f'SLEA_SB{i}min'] = MIN_MAXAU['time_ini'] - MIN_MAXAU['time_ini'] + dfmin
    MIN_MAXAU[f'SLEA_SB{i}max'] = MIN_MAXAU['time_ini'] - MIN_MAXAU['time_ini'] + dfmax
    MIN_MAXAU[f'SLEA_SB{i}mean'] = MIN_MAXAU['time_ini'] - MIN_MAXAU['time_ini'] + dfmean

In [ ]:
basf = np.arange(0,4,1)
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEAU[f'SLEA_B{i}_1']), 'M2': np.array(FINAL_FILEAU[f'SLEA_B{i}_2']), 'M3': np.array(FINAL_FILEAU[f'SLEA_B{i}_3']), 'M4': np.array(FINAL_FILEAU[f'SLEA_B{i}_4']), 'M5': np.array(FINAL_FILEAU[f'SLEA_B{i}_5']), 'M6': np.array(FINAL_FILEAU[f'SLEA_B{i}_6']), 'M7': np.array(FINAL_FILEAU[f'SLEA_B{i}_7']), 'M8': np.array(FINAL_FILEAU[f'SLEA_B{i}_8']), 'M9': np.array(FINAL_FILEAU[f'SLEA_B{i}_9']), 'M10': np.array(FINAL_FILEAU[f'SLEA_B{i}_10']), 'M11': np.array(FINAL_FILEAU[f'SLEA_B{i}_11']), 'M12': np.array(FINAL_FILEAU[f'SLEA_B{i}_12']), 'M13': np.array(FINAL_FILEAU[f'SLEA_B{i}_13']), 'M16': np.array(FINAL_FILEAU[f'SLEA_B{i}_16']), 'M17': np.array(FINAL_FILEAU[f'SLEA_B{i}_17']), 'M18': np.array(FINAL_FILEAU[f'SLEA_B{i}_18']), 'M19': np.array(FINAL_FILEAU[f'SLEA_B{i}_19'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEAU100[f'SLEA_B{i}']= FINAL_FILEAU100['memb'] - FINAL_FILEAU100['memb'] + df2100
    MIN_MAXAU[f'SLEA_B{i}min'] = MIN_MAXAU['time_ini'] - MIN_MAXAU['time_ini'] + dfmin
    MIN_MAXAU[f'SLEA_B{i}max'] = MIN_MAXAU['time_ini'] - MIN_MAXAU['time_ini'] + dfmax
    MIN_MAXAU[f'SLEA_B{i}mean'] = MIN_MAXAU['time_ini'] - MIN_MAXAU['time_ini'] + dfmean

In [ ]:
FINAL_FILEAU100.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/SLE2100_ATMO_UKESM_V2.nc')
MIN_MAXAU.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/SLE_minmax_ATMO_UKESM_V2.nc')

### ocean and both contribution

In [ ]:
# available members
membou = np.array([1,2,4,8])
                  
# bassins
subbas = np.arange(1,19,1)
bas = np.arange(1,4,1)                

#data
netCDF_fileINI = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/ismip6_states_ismip6_w1-histae_002.nc'
RESINI = xr.open_mfdataset(netCDF_fileINI)

netCDF_filemask = '/Users/jcaillet/Documents/Elmer/Papier_ete/Mask_Regions_IMBIE_ElmerGrid.nc'
mask = xr.open_mfdataset(netCDF_filemask)

netCDF_filemasks = '/Users/jcaillet/Documents/Elmer/Papier_ete/Mask_SubRegions_IMBIE_ElmerGrid.nc'
maskSS = xr.open_mfdataset(netCDF_filemasks)

#initialisation
RESINI['time']=np.arange(2006,2016,1)
data_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/ismip6_states_ismip6_w1-ukesmm1*.nc'
RES = xr.open_mfdataset(data_file)
t = np.arange(2015,2101,1)
FINAL_FILEU = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

#boucle sur membres
for j in membou:
    data_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/ismip6_states_ismip6_w1-ukesmm{j}*.nc'
    RES = xr.open_mfdataset(data_file)
    RES['time'] = np.arange(2015,2101,1)
    # boucle sur bassins
    for i in subbas:
        #ref
        RESINI[f'Vaf_SB{i}'] = (RESINI['lithkaf'] * RESINI['true_cell_area']).where(maskSS['subbasins']==i).sum('nmesh2D_face')
        RESINI[f'SLE_SB{i}'] = RESINI[f'Vaf_SB{i}'] * 1 / A_ocean * rho_ice / rho_water
        #projection
        FINAL_FILEU[f'Vaf_SB{i}_{j}'] = (RES['lithkaf'] * RES['true_cell_area']).where(maskSS['subbasins']==i).sum('nmesh2D_face')
        FINAL_FILEU[f'SLEO_SB{i}_{j}'] = -((FINAL_FILEU[f'Vaf_SB{i}_{j}'] * 1 / A_ocean * rho_ice / rho_water) - RESINI[f'SLE_SB{i}'][9]) - FINAL_FILEAU[f'SLEA_SB{i}_1']
        FINAL_FILEU[f'SLEtot_SB{i}_{j}'] = FINAL_FILEU[f'SLEO_SB{i}_{j}'] + FINAL_FILEAU[f'SLEA_SB{i}_{j}']
    for i in bas:
        ##ref
        RESINI[f'Vaf_B{i}'] = (RESINI['lithkaf'] * RESINI['true_cell_area']).where(mask['basinsIM']==i).sum('nmesh2D_face')
        RESINI[f'SLE_B{i}'] = RESINI[f'Vaf_B{i}'] * 1 / A_ocean * rho_ice / rho_water
        ##projection
        ### volume above flotation
        FINAL_FILEU[f'Vaf_B{i}_{j}'] = (RES['lithkaf'] * RES['true_cell_area']).where(mask['basinsIM']==i).sum('nmesh2D_face')
        ### SLE ocean (total - atmospheric member1)
        FINAL_FILEU[f'SLEO_B{i}_{j}'] = -((FINAL_FILEU[f'Vaf_B{i}_{j}'] * 1 / A_ocean * rho_ice / rho_water) - RESINI[f'SLE_B{i}'][9]) - FINAL_FILEAU[f'SLEA_B{i}_1']
        ### SLE total (SLE ocean + SLE atmo for the same member)
        FINAL_FILEU[f'SLEtot_B{i}_{j}'] = FINAL_FILEU[f'SLEO_B{i}_{j}'] + FINAL_FILEAU[f'SLEA_B{i}_{j}']
    for i in np.array([0]):
        #ref
        RESINI[f'Vaf_B{i}'] = (RESINI['lithkaf'] * RESINI['true_cell_area']).sum('nmesh2D_face')
        RESINI[f'SLE_B{i}'] = RESINI[f'Vaf_B{i}'] * 1 / A_ocean * rho_ice / rho_water
        #projection
        FINAL_FILEU[f'Vaf_B{i}_{j}'] = (RES['lithkaf'] * RES['true_cell_area']).sum('nmesh2D_face')
        FINAL_FILEU[f'SLEO_B{i}_{j}'] = -((FINAL_FILEU[f'Vaf_B{i}_{j}'] * 1 / A_ocean * rho_ice / rho_water) - RESINI[f'SLE_B{i}'][9]) - FINAL_FILEAU[f'SLEA_B{i}_1']
        FINAL_FILEU[f'SLEtot_B{i}_{j}'] = FINAL_FILEU[f'SLEO_B{i}_{j}'] + FINAL_FILEAU[f'SLEA_B{i}_{j}']

In [ ]:
FINAL_FILETU100 = xr.Dataset(data_vars=dict(memb=(["member"], membou)), coords=dict(member=membou))
t = np.arange(2015,2101,1)
MIN_MAXTU = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

for i in subbas:
    print(i)
    d = {'M1': np.array(FINAL_FILEU[f'SLEtot_SB{i}_1']), 'M2': np.array(FINAL_FILEU[f'SLEtot_SB{i}_2']), 'M4': np.array(FINAL_FILEU[f'SLEtot_SB{i}_4']), 'M8': np.array(FINAL_FILEU[f'SLEtot_SB{i}_8'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILETU100[f'SLEtot_SB{i}']= FINAL_FILETU100['memb'] - FINAL_FILETU100['memb'] + df2100
    MIN_MAXTU[f'SLEtot_SB{i}min'] = MIN_MAXTU['time_ini'] - MIN_MAXTU['time_ini'] + dfmin
    MIN_MAXTU[f'SLEtot_SB{i}max'] = MIN_MAXTU['time_ini'] - MIN_MAXTU['time_ini'] + dfmax
    MIN_MAXTU[f'SLEtot_SB{i}mean'] = MIN_MAXTU['time_ini'] - MIN_MAXTU['time_ini'] + dfmean

In [ ]:
basf = np.arange(0,4,1)
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEU[f'SLEtot_B{i}_1']), 'M2': np.array(FINAL_FILEU[f'SLEtot_B{i}_2']), 'M4': np.array(FINAL_FILEU[f'SLEtot_B{i}_4']), 'M8': np.array(FINAL_FILEU[f'SLEtot_B{i}_8'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILETU100[f'SLEtot_B{i}']= FINAL_FILETU100['memb'] - FINAL_FILETU100['memb'] + df2100
    MIN_MAXTU[f'SLEtot_B{i}min'] = MIN_MAXTU['time_ini'] - MIN_MAXTU['time_ini'] + dfmin
    MIN_MAXTU[f'SLEtot_B{i}max'] = MIN_MAXTU['time_ini'] - MIN_MAXTU['time_ini'] + dfmax
    MIN_MAXTU[f'SLEtot_B{i}mean'] = MIN_MAXTU['time_ini'] - MIN_MAXTU['time_ini'] + dfmean

In [ ]:
t = np.arange(2015,2101,1)
MIN_MAXTU = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

d = {'M1': np.array(FINAL_FILEU[f'SLEtot_B0_1']), 'M2': np.array(FINAL_FILEU[f'SLEtot_B0_2']), 'M4': np.array(FINAL_FILEU[f'SLEtot_B0_4']), 'M8': np.array(FINAL_FILEU[f'SLEtot_B0_8'])}
df = pd.DataFrame(data=d)
dfstd = np.array(df.std(axis = 1))
dfmean = np.array(df.mean(axis = 1))
MIN_MAXTU[f'SLEtot_B0std'] = MIN_MAXTU['time_ini'] - MIN_MAXTU['time_ini'] + dfstd
MIN_MAXTU[f'SLEtot_B0mean'] = MIN_MAXTU['time_ini'] - MIN_MAXTU['time_ini'] + dfmean

In [ ]:
FINAL_FILETU100.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/SLE2100_TOT_UKESM_V3.nc')
MIN_MAXTU.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/SLE_minmax_TOT_UKESM_V3.nc')

In [ ]:
FINAL_FILEOU100 = xr.Dataset(data_vars=dict(memb=(["member"], membou)), coords=dict(member=membou))
t = np.arange(2015,2101,1)
MIN_MAXOU = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

for i in subbas:
    print(i)
    d = {'M1': np.array(FINAL_FILEU[f'SLEO_SB{i}_1']), 'M2': np.array(FINAL_FILEU[f'SLEO_SB{i}_2']), 'M4': np.array(FINAL_FILEU[f'SLEO_SB{i}_4']), 'M8': np.array(FINAL_FILEU[f'SLEO_SB{i}_8'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEOU100[f'SLEO_SB{i}']= FINAL_FILEOU100['memb'] - FINAL_FILEOU100['memb'] + df2100
    MIN_MAXOU[f'SLEO_SB{i}min'] = MIN_MAXOU['time_ini'] - MIN_MAXOU['time_ini'] + dfmin
    MIN_MAXOU[f'SLEO_SB{i}max'] = MIN_MAXOU['time_ini'] - MIN_MAXOU['time_ini'] + dfmax
    MIN_MAXOU[f'SLEO_SB{i}mean'] = MIN_MAXOU['time_ini'] - MIN_MAXOU['time_ini'] + dfmean

In [ ]:

for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEU[f'SLEO_B{i}_1']), 'M2': np.array(FINAL_FILEU[f'SLEO_B{i}_2']), 'M4': np.array(FINAL_FILEU[f'SLEO_B{i}_4']), 'M8': np.array(FINAL_FILEU[f'SLEO_B{i}_8'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEOU100[f'SLEO_B{i}']= FINAL_FILEOU100['memb'] - FINAL_FILEOU100['memb'] + df2100
    MIN_MAXOU[f'SLEO_B{i}min'] = MIN_MAXOU['time_ini'] - MIN_MAXOU['time_ini'] + dfmin
    MIN_MAXOU[f'SLEO_B{i}max'] = MIN_MAXOU['time_ini'] - MIN_MAXOU['time_ini'] + dfmax
    MIN_MAXOU[f'SLEO_B{i}mean'] = MIN_MAXOU['time_ini'] - MIN_MAXOU['time_ini'] + dfmean

In [ ]:
FINAL_FILEOU100.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/SLE2100_OC_UKESM_V3.nc')
MIN_MAXOU.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/SLE_minmax_OC_UKESM_V3.nc')

## IPSL

### atmospheric

In [ ]:
# Open data 
netCDF_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/MAR-IPSL-CM6A-LR-r1i1p1f1_asmb_2015-2100_ssp245_regrid_04000m_FROM_IPSL-CM6A-LR-r1i1p1f1-ssp585.nc'
ATMOI = xr.open_mfdataset(netCDF_file, decode_times=False)

netCDF_filemask = '/Users/jcaillet/Documents/Elmer/DATA/BASINS/Mask_SubRegions_IMBIE.nc'
maskS = xr.open_mfdataset(netCDF_filemask)

netCDF_filemask = '/Users/jcaillet/Documents/Elmer/DATA/BASINS/Mask_Regions_IMBIE.nc'
maskR = xr.open_mfdataset(netCDF_filemask)

netCDF_ini = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/RCM_ice_regrid_04000m.nc2'
ini = xr.open_mfdataset(netCDF_ini)

# available members
membi = np.array([1,2,3,4,5,6,10,11,14,22,25])

# bassins
subbas = np.arange(1,19,1)
bas = np.arange(1,4,1)
basf = np.arange(0,4,1)

# interpolation of mask on the same grid as MAR data
ATMOI['x']=ini['x']
ATMOI['y']=ini['y']
maskS_interp = maskS.interp_like(ATMOI)
maskR_interp = maskR.interp_like(ATMOI)

#initialisation
t = np.arange(2015,2101,1)
FINAL_FILEAI = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

#boucle sur membres
for j in membi :
    data_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/MAR-IPSL-CM6A-LR-r{j}i1p1f*_asmb_2015-2100_ssp245_regrid_04000m_FROM_IPSL-CM6A-LR-r1i1p1f1-ssp585.nc'
    RES = xr.open_mfdataset(data_file, decode_times=False)
    RES['time']=FINAL_FILEAI['time']
    # boucle sur bassins
    for i in subbas:
        FINAL_FILEAI[f'SLEA_SB{i}_{j}'] = (RES['asmb'] * fac * ini['af2'] * ini['GROUND']).where(maskS['SubBasins_extrap']==i).sum(['x','y'])
        FINAL_FILEAI[f'SLEA_SB{i}_{j}'] = -(np.cumsum(FINAL_FILEAI[f'SLEA_SB{i}_{j}']) * 1 / (A_ocean * rho_water))
    for i in bas:
        FINAL_FILEAI[f'SLEA_B{i}_{j}'] = (RES['asmb'] * fac * ini['af2'] * ini['GROUND']).where(maskR['Basins_extrap']==i).sum(['x','y'])
        FINAL_FILEAI[f'SLEA_B{i}_{j}'] = -(np.cumsum(FINAL_FILEAI[f'SLEA_B{i}_{j}']) * 1 / (A_ocean * rho_water))       
    for i in np.array([0]):
        #projection
        FINAL_FILEAI[f'SLEA_B{i}_{j}'] = (RES['asmb'] * fac * ini['af2'] * ini['GROUND']).sum(['x','y'])
        FINAL_FILEAI[f'SLEA_B{i}_{j}'] = -(np.cumsum(FINAL_FILEAI[f'SLEA_B{i}_{j}']) * 1 / (A_ocean * rho_water))
# SLE en m !

In [ ]:
FINAL_FILEAI100 = xr.Dataset(data_vars=dict(memb=(["member"], membi)), coords=dict(member=membi))
t = np.arange(2015,2101,1)
MIN_MAXAI = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

for i in subbas:
    print(i)
    d = {'M1': np.array(FINAL_FILEAI[f'SLEA_SB{i}_1']), 'M2': np.array(FINAL_FILEAI[f'SLEA_SB{i}_2']), 'M3': np.array(FINAL_FILEAI[f'SLEA_SB{i}_3']), 'M4': np.array(FINAL_FILEAI[f'SLEA_SB{i}_4']), 'M5': np.array(FINAL_FILEAI[f'SLEA_SB{i}_5']), 'M6': np.array(FINAL_FILEAI[f'SLEA_SB{i}_6']), 'M10': np.array(FINAL_FILEAI[f'SLEA_SB{i}_10']), 'M11': np.array(FINAL_FILEAI[f'SLEA_SB{i}_11']), 'M14': np.array(FINAL_FILEAI[f'SLEA_SB{i}_14']), 'M22': np.array(FINAL_FILEAI[f'SLEA_SB{i}_22']), 'M25': np.array(FINAL_FILEAI[f'SLEA_SB{i}_25'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEAI100[f'SLEA_SB{i}']= FINAL_FILEAI100['memb'] - FINAL_FILEAI100['memb'] + df2100
    MIN_MAXAI[f'SLEA_SB{i}min'] = MIN_MAXAI['time_ini'] - MIN_MAXAI['time_ini'] + dfmin
    MIN_MAXAI[f'SLEA_SB{i}max'] = MIN_MAXAI['time_ini'] - MIN_MAXAI['time_ini'] + dfmax
    MIN_MAXAI[f'SLEA_SB{i}mean'] = MIN_MAXAI['time_ini'] - MIN_MAXAI['time_ini'] + dfmean

In [ ]:
basf = np.arange(0,4,1)
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEAI[f'SLEA_B{i}_1']), 'M2': np.array(FINAL_FILEAI[f'SLEA_B{i}_2']), 'M3': np.array(FINAL_FILEAI[f'SLEA_B{i}_3']), 'M4': np.array(FINAL_FILEAI[f'SLEA_B{i}_4']), 'M5': np.array(FINAL_FILEAI[f'SLEA_B{i}_5']), 'M6': np.array(FINAL_FILEAI[f'SLEA_B{i}_6']), 'M10': np.array(FINAL_FILEAI[f'SLEA_B{i}_10']), 'M11': np.array(FINAL_FILEAI[f'SLEA_B{i}_11']), 'M14': np.array(FINAL_FILEAI[f'SLEA_B{i}_14']), 'M22': np.array(FINAL_FILEAI[f'SLEA_B{i}_22']), 'M25': np.array(FINAL_FILEAI[f'SLEA_B{i}_25'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEAI100[f'SLEA_B{i}']= FINAL_FILEAI100['memb'] - FINAL_FILEAI100['memb'] + df2100
    MIN_MAXAI[f'SLEA_B{i}min'] = MIN_MAXAI['time_ini'] - MIN_MAXAI['time_ini'] + dfmin
    MIN_MAXAI[f'SLEA_B{i}max'] = MIN_MAXAI['time_ini'] - MIN_MAXAI['time_ini'] + dfmax
    MIN_MAXAI[f'SLEA_B{i}mean'] = MIN_MAXAI['time_ini'] - MIN_MAXAI['time_ini'] + dfmean

In [ ]:
FINAL_FILEAI100.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/SLE2100_ATMO_IPSL_V2.nc')
MIN_MAXAI.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/SLE_minmax_ATMO_IPSL_V2.nc')

### oceanic and both

In [ ]:
# available members
memboi = np.array([1,3,6,11,25])
                  
# bassins
subbas = np.arange(1,19,1)
bas = np.arange(1,4,1)                

#data
netCDF_fileINI = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/ismip6_states_ismip6_w1-histae_002.nc'
RESINI = xr.open_mfdataset(netCDF_fileINI)

netCDF_filemask = '/Users/jcaillet/Documents/Elmer/Papier_ete/Mask_Regions_IMBIE_ElmerGrid.nc'
mask = xr.open_mfdataset(netCDF_filemask)

netCDF_filemasks = '/Users/jcaillet/Documents/Elmer/Papier_ete/Mask_SubRegions_IMBIE_ElmerGrid.nc'
maskSS = xr.open_mfdataset(netCDF_filemasks)

#initialisation
RESINI['time']=np.arange(2006,2016,1)
data_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/ismip6_states_ismip6_w1-ipslm1_*.nc'
RES = xr.open_mfdataset(data_file)
t = np.arange(2015,2101,1)
FINAL_FILEI = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

#boucle sur membres
for j in memboi:
    data_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/ismip6_states_ismip6_w1-ipslm{j}_*.nc'
    RES = xr.open_mfdataset(data_file)
    RES['time'] = np.arange(2015,2101,1)
    # boucle sur bassins
    for i in subbas:
        #ref
        RESINI[f'Vaf_SB{i}'] = (RESINI['lithkaf'] * RESINI['true_cell_area']).where(maskSS['subbasins']==i).sum('nmesh2D_face')
        RESINI[f'SLE_SB{i}'] = RESINI[f'Vaf_SB{i}'] * 1 / A_ocean * rho_ice / rho_water
        #projection
        FINAL_FILEI[f'Vaf_SB{i}_{j}'] = (RES['lithkaf'] * RES['true_cell_area']).where(maskSS['subbasins']==i).sum('nmesh2D_face')       
        FINAL_FILEI[f'SLEO_SB{i}_{j}'] = -((FINAL_FILEI[f'Vaf_SB{i}_{j}'] * 1 / A_ocean * rho_ice / rho_water) - RESINI[f'SLE_SB{i}'][9]) - FINAL_FILEAI[f'SLEA_SB{i}_1']
        FINAL_FILEI[f'SLEtot_SB{i}_{j}'] = FINAL_FILEI[f'SLEO_SB{i}_{j}'] + FINAL_FILEAI[f'SLEA_SB{i}_{j}']
    for i in bas:
        ##ref
        RESINI[f'Vaf_B{i}'] = (RESINI['lithkaf'] * RESINI['true_cell_area']).where(mask['basinsIM']==i).sum('nmesh2D_face')
        RESINI[f'SLE_B{i}'] = RESINI[f'Vaf_B{i}'] * 1 / A_ocean * rho_ice / rho_water
        ##projection
        FINAL_FILEI[f'Vaf_B{i}_{j}'] = (RES['lithkaf'] * RES['true_cell_area']).where(mask['basinsIM']==i).sum('nmesh2D_face')
        FINAL_FILEI[f'SLEO_B{i}_{j}'] = -((FINAL_FILEI[f'Vaf_B{i}_{j}'] * 1 / A_ocean * rho_ice / rho_water) - RESINI[f'SLE_B{i}'][9]) - FINAL_FILEAI[f'SLEA_B{i}_1']
        FINAL_FILEI[f'SLEtot_B{i}_{j}'] = FINAL_FILEI[f'SLEO_B{i}_{j}'] + FINAL_FILEAI[f'SLEA_B{i}_{j}']
    for i in np.array([0]):
        #ref
        RESINI[f'Vaf_B{i}'] = (RESINI['lithkaf'] * RESINI['true_cell_area']).sum('nmesh2D_face')
        RESINI[f'SLE_B{i}'] = RESINI[f'Vaf_B{i}'] * 1 / A_ocean * rho_ice / rho_water
        #projection
        FINAL_FILEI[f'Vaf_B{i}_{j}'] = (RES['lithkaf'] * RES['true_cell_area']).sum('nmesh2D_face')
        FINAL_FILEI[f'SLEO_B{i}_{j}'] = -((FINAL_FILEI[f'Vaf_B{i}_{j}'] * 1 / A_ocean * rho_ice / rho_water) - RESINI[f'SLE_B{i}'][9]) - FINAL_FILEAI[f'SLEA_B{i}_1']
        FINAL_FILEI[f'SLEtot_B{i}_{j}'] = FINAL_FILEI[f'SLEO_B{i}_{j}'] + FINAL_FILEAI[f'SLEA_B{i}_{j}']

In [ ]:
FINAL_FILETI100 = xr.Dataset(data_vars=dict(memb=(["member"], memboi)), coords=dict(member=memboi))
t = np.arange(2015,2101,1)
MIN_MAXTI = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

for i in subbas:
    print(i)
    d = {'M1': np.array(FINAL_FILEI[f'SLEtot_SB{i}_1']), 'M3': np.array(FINAL_FILEI[f'SLEtot_SB{i}_3']), 'M6': np.array(FINAL_FILEI[f'SLEtot_SB{i}_6']), 'M11': np.array(FINAL_FILEI[f'SLEtot_SB{i}_11']), 'M25': np.array(FINAL_FILEI[f'SLEtot_SB{i}_25'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILETI100[f'SLEtot_SB{i}']= FINAL_FILETI100['memb'] - FINAL_FILETI100['memb'] + df2100
    MIN_MAXTI[f'SLEtot_SB{i}min'] = MIN_MAXTI['time_ini'] - MIN_MAXTI['time_ini'] + dfmin
    MIN_MAXTI[f'SLEtot_SB{i}max'] = MIN_MAXTI['time_ini'] - MIN_MAXTI['time_ini'] + dfmax
    MIN_MAXTI[f'SLEtot_SB{i}mean'] = MIN_MAXTI['time_ini'] - MIN_MAXTI['time_ini'] + dfmean

In [ ]:
basf = np.arange(0,4,1)
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEI[f'SLEtot_B{i}_1']), 'M3': np.array(FINAL_FILEI[f'SLEtot_B{i}_3']), 'M6': np.array(FINAL_FILEI[f'SLEtot_B{i}_6']), 'M11': np.array(FINAL_FILEI[f'SLEtot_B{i}_11']), 'M25': np.array(FINAL_FILEI[f'SLEtot_B{i}_25'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILETI100[f'SLEtot_B{i}']= FINAL_FILETI100['memb'] - FINAL_FILETI100['memb'] + df2100
    MIN_MAXTI[f'SLEtot_B{i}min'] = MIN_MAXTI['time_ini'] - MIN_MAXTI['time_ini'] + dfmin
    MIN_MAXTI[f'SLEtot_B{i}max'] = MIN_MAXTI['time_ini'] - MIN_MAXTI['time_ini'] + dfmax
    MIN_MAXTI[f'SLEtot_B{i}mean'] = MIN_MAXTI['time_ini'] - MIN_MAXTI['time_ini'] + dfmean

In [ ]:
t = np.arange(2015,2101,1)
MIN_MAXTI = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))
basf = np.arange(0,1,1)
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEI[f'SLEtot_B{i}_1']), 'M3': np.array(FINAL_FILEI[f'SLEtot_B{i}_3']), 'M6': np.array(FINAL_FILEI[f'SLEtot_B{i}_6']), 'M11': np.array(FINAL_FILEI[f'SLEtot_B{i}_11']), 'M25': np.array(FINAL_FILEI[f'SLEtot_B{i}_25'])}
    df = pd.DataFrame(data=d)
    dfstd = np.array(df.std(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    MIN_MAXTI[f'SLEtot_B{i}std'] = MIN_MAXTI['time_ini'] - MIN_MAXTI['time_ini'] + dfstd
    MIN_MAXTI[f'SLEtot_B{i}mean'] = MIN_MAXTI['time_ini'] - MIN_MAXTI['time_ini'] + dfmean

In [ ]:
FINAL_FILETI100.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/SLE2100_TOT_IPSL_V3.nc')
MIN_MAXTI.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/SLE_minmax_TOT_IPSL_V3.nc')

In [ ]:
FINAL_FILEOI100 = xr.Dataset(data_vars=dict(memb=(["member"], memboi)), coords=dict(member=memboi))
t = np.arange(2015,2101,1)
MIN_MAXOI = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

for i in subbas:
    print(i)
    d = {'M1': np.array(FINAL_FILEI[f'SLEO_SB{i}_1']), 'M3': np.array(FINAL_FILEI[f'SLEO_SB{i}_3']), 'M6': np.array(FINAL_FILEI[f'SLEO_SB{i}_6']), 'M11': np.array(FINAL_FILEI[f'SLEO_SB{i}_11']), 'M25': np.array(FINAL_FILEI[f'SLEO_SB{i}_25'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEOI100[f'SLEO_SB{i}']= FINAL_FILEOI100['memb'] - FINAL_FILEOI100['memb'] + df2100
    MIN_MAXOI[f'SLEO_SB{i}min'] = MIN_MAXOI['time_ini'] - MIN_MAXOI['time_ini'] + dfmin
    MIN_MAXOI[f'SLEO_SB{i}max'] = MIN_MAXOI['time_ini'] - MIN_MAXOI['time_ini'] + dfmax
    MIN_MAXOI[f'SLEO_SB{i}mean'] = MIN_MAXOI['time_ini'] - MIN_MAXOI['time_ini'] + dfmean

In [ ]:
basf = np.arange(0,4,1)
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEI[f'SLEO_B{i}_1']), 'M3': np.array(FINAL_FILEI[f'SLEO_B{i}_3']), 'M6': np.array(FINAL_FILEI[f'SLEO_B{i}_6']), 'M11': np.array(FINAL_FILEI[f'SLEO_B{i}_11']), 'M25': np.array(FINAL_FILEI[f'SLEO_B{i}_25'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEOI100[f'SLEO_B{i}']= FINAL_FILEOI100['memb'] - FINAL_FILEOI100['memb'] + df2100
    MIN_MAXOI[f'SLEO_B{i}min'] = MIN_MAXOI['time_ini'] - MIN_MAXOI['time_ini'] + dfmin
    MIN_MAXOI[f'SLEO_B{i}max'] = MIN_MAXOI['time_ini'] - MIN_MAXOI['time_ini'] + dfmax
    MIN_MAXOI[f'SLEO_B{i}mean'] = MIN_MAXOI['time_ini'] - MIN_MAXOI['time_ini'] + dfmean

In [ ]:
FINAL_FILEOI100.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/SLE2100_OC_IPSL_V3.nc')
MIN_MAXOI.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/SLE_minmax_OC_IPSL_V3.nc')

## MPI

### atmospheric

In [ ]:
# Open data 
netCDF_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/MAR-MPI-ESM1-2-HR-r1i1p1f1_asmb_2015-2100_ssp245_regrid_04000m_FROM_MPI-ESM1-2-HR-r1i1p1f1-ssp245.nc'
ATMOM = xr.open_mfdataset(netCDF_file, decode_times=False)

netCDF_filemask = '/Users/jcaillet/Documents/Elmer/DATA/BASINS/Mask_SubRegions_IMBIE.nc'
maskS = xr.open_mfdataset(netCDF_filemask)

netCDF_filemask = '/Users/jcaillet/Documents/Elmer/DATA/BASINS/Mask_Regions_IMBIE.nc'
maskR = xr.open_mfdataset(netCDF_filemask)

netCDF_ini = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/RCM_ice_regrid_04000m.nc2'
ini = xr.open_mfdataset(netCDF_ini)

# available members
membm = np.array([1,2])

# bassins
subbas = np.arange(1,19,1)
bas = np.arange(1,4,1)
basf = np.arange(0,4,1)

# interpolation of mask on the same grid as MAR data
ATMOM['x']=ini['x']
ATMOM['y']=ini['y']
maskS_interp = maskS.interp_like(ATMOM)
maskR_interp = maskR.interp_like(ATMOM)

#initialisation
t = np.arange(2015,2101,1)
FINAL_FILEAM = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

#boucle sur membres
for j in membm :
    data_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/MAR-MPI-ESM1-2-HR-r{j}i1p1f*_asmb_2015-2100_ssp245_regrid_04000m_FROM_MPI-ESM1-2-HR-r1i1p1f1-ssp245.nc'
    RES = xr.open_mfdataset(data_file, decode_times=False)
    RES['time']=FINAL_FILEAM['time']
    # boucle sur bassins
    for i in subbas:
        FINAL_FILEAM[f'SLEA_SB{i}_{j}'] = (RES['asmb'] * fac * ini['af2'] * ini['GROUND']).where(maskS['SubBasins_extrap']==i).sum(['x','y'])
        FINAL_FILEAM[f'SLEA_SB{i}_{j}'] = -(np.cumsum(FINAL_FILEAM[f'SLEA_SB{i}_{j}']) * 1 / (A_ocean * rho_water))
    for i in bas:
        FINAL_FILEAM[f'SLEA_B{i}_{j}'] = (RES['asmb'] * fac * ini['af2'] * ini['GROUND']).where(maskR['Basins_extrap']==i).sum(['x','y'])
        FINAL_FILEAM[f'SLEA_B{i}_{j}'] = -(np.cumsum(FINAL_FILEAM[f'SLEA_B{i}_{j}']) * 1 / (A_ocean * rho_water))       
    for i in np.array([0]):
        #projection
        FINAL_FILEAM[f'SLEA_B{i}_{j}'] = (RES['asmb'] * fac * ini['af2'] * ini['GROUND']).sum(['x','y'])
        FINAL_FILEAM[f'SLEA_B{i}_{j}'] = -(np.cumsum(FINAL_FILEAM[f'SLEA_B{i}_{j}']) * 1 / (A_ocean * rho_water))
# SLE en m !

In [ ]:
FINAL_FILEAM100 = xr.Dataset(data_vars=dict(memb=(["member"], membm)), coords=dict(member=membm))
t = np.arange(2015,2101,1)
MIN_MAXAM = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

for i in subbas:
    print(i)
    d = {'M1': np.array(FINAL_FILEAM[f'SLEA_SB{i}_1']), 'M2': np.array(FINAL_FILEAM[f'SLEA_SB{i}_2'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEAM100[f'SLEA_SB{i}']= FINAL_FILEAM100['memb'] - FINAL_FILEAM100['memb'] + df2100
    MIN_MAXAM[f'SLEA_SB{i}min'] = MIN_MAXAM['time_ini'] - MIN_MAXAM['time_ini'] + dfmin
    MIN_MAXAM[f'SLEA_SB{i}max'] = MIN_MAXAM['time_ini'] - MIN_MAXAM['time_ini'] + dfmax
    MIN_MAXAM[f'SLEA_SB{i}mean'] = MIN_MAXAM['time_ini'] - MIN_MAXAM['time_ini'] + dfmean

In [ ]:
basf = np.arange(0,4,1)
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEAM[f'SLEA_B{i}_1']), 'M2': np.array(FINAL_FILEAM[f'SLEA_B{i}_2'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEAM100[f'SLEA_B{i}']= FINAL_FILEAM100['memb'] - FINAL_FILEAM100['memb'] + df2100
    MIN_MAXAM[f'SLEA_B{i}min'] = MIN_MAXAM['time_ini'] - MIN_MAXAM['time_ini'] + dfmin
    MIN_MAXAM[f'SLEA_B{i}max'] = MIN_MAXAM['time_ini'] - MIN_MAXAM['time_ini'] + dfmax
    MIN_MAXAM[f'SLEA_B{i}mean'] = MIN_MAXAM['time_ini'] - MIN_MAXAM['time_ini'] + dfmean

In [ ]:
FINAL_FILEAM100.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/SLE2100_ATMO_MPI_V2.nc')
MIN_MAXAM.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/SLE_minmax_ATMO_MPI_V2.nc')

### oceanic and both contribution

In [ ]:
# available members
membom = np.array([1,2])
                  
# bassins
subbas = np.arange(1,19,1)
bas = np.arange(1,4,1)                

#data
netCDF_fileINI = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/ismip6_states_ismip6_w1-histae_002.nc'
RESINI = xr.open_mfdataset(netCDF_fileINI)

netCDF_filemask = '/Users/jcaillet/Documents/Elmer/Papier_ete/Mask_Regions_IMBIE_ElmerGrid.nc'
mask = xr.open_mfdataset(netCDF_filemask)

netCDF_filemasks = '/Users/jcaillet/Documents/Elmer/Papier_ete/Mask_SubRegions_IMBIE_ElmerGrid.nc'
maskSS = xr.open_mfdataset(netCDF_filemasks)

#initialisation
RESINI['time']=np.arange(2006,2016,1)
data_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/ismip6_states_ismip6_w1-mpim1*.nc'
RES = xr.open_mfdataset(data_file)
t = np.arange(2015,2101,1)
FINAL_FILEM = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

#boucle sur membres
for j in membom:
    data_file = f'/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/ismip6_states_ismip6_w1-mpim{j}_*.nc'
    RES = xr.open_mfdataset(data_file)
    RES['time'] = np.arange(2015,2101,1)
    # boucle sur bassins
    for i in subbas:
        #ref
        RESINI[f'Vaf_SB{i}'] = (RESINI['lithkaf'] * RESINI['true_cell_area']).where(maskSS['subbasins']==i).sum('nmesh2D_face')
        RESINI[f'SLE_SB{i}'] = RESINI[f'Vaf_SB{i}'] * 1 / A_ocean * rho_ice / rho_water
        #projection
        FINAL_FILEM[f'Vaf_SB{i}_{j}'] = (RES['lithkaf'] * RES['true_cell_area']).where(maskSS['subbasins']==i).sum('nmesh2D_face')
        FINAL_FILEM[f'SLEO_SB{i}_{j}'] = -((FINAL_FILEM[f'Vaf_SB{i}_{j}'] * 1 / A_ocean * rho_ice / rho_water) - RESINI[f'SLE_SB{i}'][9]) - FINAL_FILEAM[f'SLEA_SB{i}_1']
        FINAL_FILEM[f'SLEtot_SB{i}_{j}'] = FINAL_FILEM[f'SLEO_SB{i}_{j}'] + FINAL_FILEAM[f'SLEA_SB{i}_{j}']
    for i in bas:
        ##ref
        RESINI[f'Vaf_B{i}'] = (RESINI['lithkaf'] * RESINI['true_cell_area']).where(mask['basinsIM']==i).sum('nmesh2D_face')
        RESINI[f'SLE_B{i}'] = RESINI[f'Vaf_B{i}'] * 1 / A_ocean * rho_ice / rho_water
        ##projection
        FINAL_FILEM[f'Vaf_B{i}_{j}'] = (RES['lithkaf'] * RES['true_cell_area']).where(mask['basinsIM']==i).sum('nmesh2D_face')
        FINAL_FILEM[f'SLEO_B{i}_{j}'] = -((FINAL_FILEM[f'Vaf_B{i}_{j}'] * 1 / A_ocean * rho_ice / rho_water) - RESINI[f'SLE_B{i}'][9]) - FINAL_FILEAM[f'SLEA_B{i}_1']
        FINAL_FILEM[f'SLEtot_B{i}_{j}'] = FINAL_FILEM[f'SLEO_B{i}_{j}'] + FINAL_FILEAM[f'SLEA_B{i}_{j}']
        
    for i in np.array([0]):
        #ref
        RESINI[f'Vaf_B{i}'] = (RESINI['lithkaf'] * RESINI['true_cell_area']).sum('nmesh2D_face')
        RESINI[f'SLE_B{i}'] = RESINI[f'Vaf_B{i}'] * 1 / A_ocean * rho_ice / rho_water
        #projection
        FINAL_FILEM[f'Vaf_B{i}_{j}'] = (RES['lithkaf'] * RES['true_cell_area']).sum('nmesh2D_face')
        FINAL_FILEM[f'SLEO_B{i}_{j}'] = -((FINAL_FILEM[f'Vaf_B{i}_{j}'] * 1 / A_ocean * rho_ice / rho_water) - RESINI[f'SLE_B{i}'][9]) - FINAL_FILEAM[f'SLEA_B{i}_1']
        FINAL_FILEM[f'SLEtot_B{i}_{j}'] = FINAL_FILEM[f'SLEO_B{i}_{j}'] + FINAL_FILEAM[f'SLEA_B{i}_{j}']

In [ ]:
FINAL_FILETM100 = xr.Dataset(data_vars=dict(memb=(["member"], membom)), coords=dict(member=membom))
t = np.arange(2015,2101,1)
MIN_MAXTM = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

for i in subbas:
    print(i)
    d = {'M1': np.array(FINAL_FILEM[f'SLEtot_SB{i}_1']), 'M2': np.array(FINAL_FILEM[f'SLEtot_SB{i}_2'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILETM100[f'SLEtot_SB{i}']= FINAL_FILETM100['memb'] - FINAL_FILETM100['memb'] + df2100
    MIN_MAXTM[f'SLEtot_SB{i}min'] = MIN_MAXTM['time_ini'] - MIN_MAXTM['time_ini'] + dfmin
    MIN_MAXTM[f'SLEtot_SB{i}max'] = MIN_MAXTM['time_ini'] - MIN_MAXTM['time_ini'] + dfmax
    MIN_MAXTM[f'SLEtot_SB{i}mean'] = MIN_MAXTM['time_ini'] - MIN_MAXTM['time_ini'] + dfmean

In [ ]:
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEM[f'SLEtot_B{i}_1']), 'M2': np.array(FINAL_FILEM[f'SLEtot_B{i}_2'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILETM100[f'SLEtot_B{i}']= FINAL_FILETM100['memb'] - FINAL_FILETM100['memb'] + df2100
    MIN_MAXTM[f'SLEtot_B{i}min'] = MIN_MAXTM['time_ini'] - MIN_MAXTM['time_ini'] + dfmin
    MIN_MAXTM[f'SLEtot_B{i}max'] = MIN_MAXTM['time_ini'] - MIN_MAXTM['time_ini'] + dfmax
    MIN_MAXTM[f'SLEtot_B{i}mean'] = MIN_MAXTM['time_ini'] - MIN_MAXTM['time_ini'] + dfmean

In [ ]:
t = np.arange(2015,2101,1)
MIN_MAXTM = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))
basf = np.arange(0,1,1) 
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEM[f'SLEtot_B{i}_1']), 'M2': np.array(FINAL_FILEM[f'SLEtot_B{i}_2'])}
    df = pd.DataFrame(data=d)
    dfstd = np.array(df.std(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    MIN_MAXTM[f'SLEtot_B{i}std'] = MIN_MAXTM['time_ini'] - MIN_MAXTM['time_ini'] + dfstd
    MIN_MAXTM[f'SLEtot_B{i}mean'] = MIN_MAXTM['time_ini'] - MIN_MAXTM['time_ini'] + dfmean

In [ ]:
FINAL_FILETM100.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/SLE2100_TOT_MPI_V3.nc')
MIN_MAXTM.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/SLE_minmax_TOT_MPI_V3.nc')

In [ ]:
FINAL_FILEOM100 = xr.Dataset(data_vars=dict(memb=(["member"], membom)), coords=dict(member=membom))
t = np.arange(2015,2101,1)
MIN_MAXOM = xr.Dataset(data_vars=dict(time_ini=(["time"], t)), coords=dict(time=t))

for i in subbas:
    print(i)
    d = {'M1': np.array(FINAL_FILEM[f'SLEO_SB{i}_1']), 'M2': np.array(FINAL_FILEM[f'SLEO_SB{i}_2'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEOM100[f'SLEO_SB{i}']= FINAL_FILEOM100['memb'] - FINAL_FILEOM100['memb'] + df2100
    MIN_MAXOM[f'SLEO_SB{i}min'] = MIN_MAXOM['time_ini'] - MIN_MAXOM['time_ini'] + dfmin
    MIN_MAXOM[f'SLEO_SB{i}max'] = MIN_MAXOM['time_ini'] - MIN_MAXOM['time_ini'] + dfmax
    MIN_MAXOM[f'SLEO_SB{i}mean'] = MIN_MAXOM['time_ini'] - MIN_MAXOM['time_ini'] + dfmean

In [ ]:
for i in basf:
    print(i)
    d = {'M1': np.array(FINAL_FILEM[f'SLEO_B{i}_1']), 'M2': np.array(FINAL_FILEM[f'SLEO_B{i}_2'])}
    df = pd.DataFrame(data=d)
    dfmin = np.array(df.min(axis = 1))
    dfmax = np.array(df.max(axis = 1))
    dfmean = np.array(df.mean(axis = 1))
    df2100 = np.array(df.iloc[85])
    FINAL_FILEOM100[f'SLEO_B{i}']= FINAL_FILEOM100['memb'] - FINAL_FILEOM100['memb'] + df2100
    MIN_MAXOM[f'SLEO_B{i}min'] = MIN_MAXOM['time_ini'] - MIN_MAXOM['time_ini'] + dfmin
    MIN_MAXOM[f'SLEO_B{i}max'] = MIN_MAXOM['time_ini'] - MIN_MAXOM['time_ini'] + dfmax
    MIN_MAXOM[f'SLEO_B{i}mean'] = MIN_MAXOM['time_ini'] - MIN_MAXOM['time_ini'] + dfmean

In [ ]:
FINAL_FILEOM100.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/SLE2100_OC_MPI_V3.nc')
MIN_MAXOM.to_netcdf('/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/SLE_minmax_OC_MPI_V3.nc')

## Figure

In [ ]:
# open netcdf file
minmax_OC_UK = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/SLE_minmax_OC_UKESM_V3.nc'
minmax_ATMO_UK = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/SLE_minmax_ATMO_UKESM_V2.nc'
minmax_TOT_UK = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/SLE_minmax_TOT_UKESM_V3.nc'
UKESM_MINMAX_OC = xr.open_mfdataset(minmax_OC_UK)
UKESM_MINMAX_ATMO = xr.open_mfdataset(minmax_ATMO_UK)
UKESM_MINMAX_TOT = xr.open_mfdataset(minmax_TOT_UK)

minmax_OC_IP = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/SLE_minmax_OC_IPSL_V3.nc'
minmax_ATMO_IP = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/SLE_minmax_ATMO_IPSL_V2.nc'
minmax_TOT_IP = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/SLE_minmax_TOT_IPSL_V3.nc'
IPSL_MINMAX_OC = xr.open_mfdataset(minmax_OC_IP)
IPSL_MINMAX_ATMO = xr.open_mfdataset(minmax_ATMO_IP)
IPSL_MINMAX_TOT = xr.open_mfdataset(minmax_TOT_IP)

minmax_OC_MP = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/SLE_minmax_OC_MPI_V3.nc'
minmax_ATMO_MP = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/SLE_minmax_ATMO_MPI_V2.nc'
minmax_TOT_MP = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/SLE_minmax_TOT_MPI_V3.nc'
MPI_MINMAX_OC = xr.open_mfdataset(minmax_OC_MP)
MPI_MINMAX_ATMO = xr.open_mfdataset(minmax_ATMO_MP)
MPI_MINMAX_TOT = xr.open_mfdataset(minmax_TOT_MP)

In [ ]:
sns.set_context('paper')

k =-3
basf = np.arange(0,4,1) # basin WAIS, EAIS,APIS,EAIS

f = plt.figure(figsize = (8, 8))
ax={}
nb_rows = 4
nb_cols = 3
#gs = gridspec.GridSpec(nb_rows,nb_cols, width_ratios=[8,1,8,1,8,1]) 
f.text(0.04, 0.5, 'Sea Level Contribution SLC (cm)', va='center', rotation='vertical')
f.text(0.5, 0.02, 'Time (year)', va='center')

plt.figtext(0.23, 0.98, 'total contribution', horizontalalignment='center', fontweight="bold")
plt.figtext(0.23, 0.96, '(ocean + atmosphere)', horizontalalignment='center', fontweight="bold")
plt.figtext(0.53, 0.98, 'dynamical contribution', horizontalalignment='center', fontweight="bold")
plt.figtext(0.53, 0.96, '(ocean)', horizontalalignment='center', fontweight="bold")
plt.figtext(0.83, 0.98, 'SMB contribution', horizontalalignment='center', fontweight="bold")
plt.figtext(0.83, 0.96, '(atmosphere)', horizontalalignment='center', fontweight="bold")
plt.figtext(0.01, 0.84, 'Entire Antarctica', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.01, 0.62, 'East Antarctica (EAIS)', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.01, 0.395, 'West Antarctica (WAIS)', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.01, 0.17, 'Antarctic Peninsula (APIS)', verticalalignment='center', rotation=90, fontweight="bold")


for i in basf :
    k = k + 3
## SLC tot
    ax[k] = f.add_subplot(nb_rows,nb_cols, k+1)
    ax[k].hlines(0, 2015, 2100, linewidth = 1.2, linestyle='--',colors= 'black')
    # UKESM
    ax[k].plot(UKESM_MINMAX_TOT.time, UKESM_MINMAX_TOT[f'SLEtot_B{i}mean']*1e2, linewidth = 1, color= 'dodgerblue', label = 'members UKESM (4)', zorder=10)
    ax[k].fill_between(UKESM_MINMAX_TOT.time, UKESM_MINMAX_TOT[f'SLEtot_B{i}min']*1e2, UKESM_MINMAX_TOT[f'SLEtot_B{i}max']*1e2, alpha = 0.2, color= 'dodgerblue', zorder=1)
    # IPSL
    ax[k].plot(IPSL_MINMAX_TOT.time, IPSL_MINMAX_TOT[f'SLEtot_B{i}mean']*1e2, linewidth = 1, color= 'darkviolet', label = 'members IPSL (5)', zorder=10)
    ax[k].fill_between(IPSL_MINMAX_TOT.time, IPSL_MINMAX_TOT[f'SLEtot_B{i}min']*1e2, IPSL_MINMAX_TOT[f'SLEtot_B{i}max']*1e2, alpha = 0.2, color= 'darkviolet', zorder=1)
    # MPI
    ax[k].plot(MPI_MINMAX_TOT.time, MPI_MINMAX_TOT[f'SLEtot_B{i}mean']*1e2, linewidth = 1, color= 'limegreen', label = 'members MPI (2)', zorder=10)
    ax[k].fill_between(MPI_MINMAX_TOT.time, MPI_MINMAX_TOT[f'SLEtot_B{i}min']*1e2, MPI_MINMAX_TOT[f'SLEtot_B{i}max']*1e2, alpha = 0.2, color= 'limegreen', zorder=1)

    if k==0:
        ax[k].set_ylim(-11,3)
        ax[k].axes.xaxis.set_ticklabels([])
        plt.legend(fontsize=7.5)
    elif k==3:
        ax[k].set_ylim(-7,1)
        ax[k].axes.xaxis.set_ticklabels([])
    elif k==6:
        ax[k].set_ylim(-4.5,4.5)
        ax[k].axes.xaxis.set_ticklabels([])
    else : 
        ax[k].set_ylim(-2.5,0.5)
    ax[k].set_xlim(2015, 2100)
    plt.grid(True)

# SLC ocean 
    ax[k+1] = f.add_subplot(nb_rows,nb_cols, k+2)
    ax[k+1].hlines(0, 2015, 2100, linewidth = 1.2, linestyle='--',colors= 'black')
    #UKESM
    ax[k+1].plot(UKESM_MINMAX_OC.time, UKESM_MINMAX_OC[f'SLEO_B{i}mean']*1e2, linewidth = 1, color= 'dodgerblue', label = 'members UKESM (4)', zorder=10)
    ax[k+1].fill_between(UKESM_MINMAX_OC.time, UKESM_MINMAX_OC[f'SLEO_B{i}min']*1e2, UKESM_MINMAX_OC[f'SLEO_B{i}max']*1e2, alpha = 0.2, color= 'dodgerblue', zorder=1)
    # IPSL
    ax[k+1].plot(IPSL_MINMAX_OC.time, IPSL_MINMAX_OC[f'SLEO_B{i}mean']*1e2, linewidth = 1, color= 'darkviolet', label = 'members IPSL (5)', zorder=10)
    ax[k+1].fill_between(IPSL_MINMAX_OC.time, IPSL_MINMAX_OC[f'SLEO_B{i}min']*1e2, IPSL_MINMAX_OC[f'SLEO_B{i}max']*1e2, alpha = 0.2, color= 'darkviolet', zorder=1)
    # MPI
    ax[k+1].plot(MPI_MINMAX_OC.time, MPI_MINMAX_OC[f'SLEO_B{i}mean']*1e2, linewidth = 1, color= 'limegreen', label = 'members MPI (2)', zorder=10)
    ax[k+1].fill_between(MPI_MINMAX_OC.time, MPI_MINMAX_OC[f'SLEO_B{i}min']*1e2, MPI_MINMAX_OC[f'SLEO_B{i}max']*1e2, alpha = 0.2, color= 'limegreen', zorder=1)

    if k==0:
        ax[k+1].set_ylim(-11,3)
        ax[k+1].axes.xaxis.set_ticklabels([])
        ax[k+1].axes.yaxis.set_ticklabels([])
        plt.legend(fontsize=7.5)
    elif k==3:
        ax[k+1].set_ylim(-7,1)
        ax[k+1].axes.xaxis.set_ticklabels([])
        ax[k+1].axes.yaxis.set_ticklabels([])
    elif k==6:
        ax[k+1].set_ylim(-4.5,4.5)
        ax[k+1].axes.xaxis.set_ticklabels([])
        ax[k+1].axes.yaxis.set_ticklabels([])
    else : 
        ax[k+1].set_ylim(-2.5,0.5)
        ax[k+1].axes.yaxis.set_ticklabels([])
    ax[k+1].set_xlim(2015, 2100)
    plt.grid(True)

# SLC atmosphere
    ax[k+2] = f.add_subplot(nb_rows,nb_cols,k+3)
    ax[k+2].hlines(0, 2015, 2100, linewidth = 1.2, linestyle='--',colors= 'black')
    # UKESM
    ax[k+2].plot(UKESM_MINMAX_ATMO.time, UKESM_MINMAX_ATMO[f'SLEA_B{i}mean']*1e2, linewidth = 1, color= 'dodgerblue', label = 'members UKESM (17)', zorder=10)
    ax[k+2].fill_between(UKESM_MINMAX_ATMO.time, UKESM_MINMAX_ATMO[f'SLEA_B{i}min']*1e2, UKESM_MINMAX_ATMO[f'SLEA_B{i}max']*1e2, alpha = 0.2, color= 'dodgerblue', zorder=1)
    # IPSL
    ax[k+2].plot(IPSL_MINMAX_ATMO.time, IPSL_MINMAX_ATMO[f'SLEA_B{i}mean']*1e2, linewidth = 1, color= 'darkviolet', label = 'members IPSL (11)', zorder=10)
    ax[k+2].fill_between(IPSL_MINMAX_ATMO.time, IPSL_MINMAX_ATMO[f'SLEA_B{i}min']*1e2, IPSL_MINMAX_ATMO[f'SLEA_B{i}max']*1e2, alpha = 0.2, color= 'darkviolet', zorder=1)
    # MPI
    ax[k+2].plot(MPI_MINMAX_ATMO.time, MPI_MINMAX_ATMO[f'SLEA_B{i}mean']*1e2, linewidth = 1, color= 'limegreen', label = 'members MPI (2)', zorder=10)
    ax[k+2].fill_between(MPI_MINMAX_ATMO.time, MPI_MINMAX_ATMO[f'SLEA_B{i}min']*1e2, MPI_MINMAX_ATMO[f'SLEA_B{i}max']*1e2, alpha = 0.2, color= 'limegreen', zorder=1)

    if k==0:
        ax[k+2].set_ylim(-11,3)
        ax[k+2].axes.xaxis.set_ticklabels([])
        plt.legend(fontsize=7.5)
    elif k==3:
        ax[k+2].set_ylim(-7,1)
        ax[k+2].axes.xaxis.set_ticklabels([])
    elif k==6:
        ax[k+2].set_ylim(-4.5,4.5)
        ax[k+2].axes.xaxis.set_ticklabels([])
    else : 
        ax[k+2].set_ylim(-2.5,0.5)
    ax[k+2].axes.yaxis.set_ticklabels([])
    ax[k+2].set_xlim(2015, 2100)
    plt.grid(True)
    
ax[0].text(0.03, 0.90, 'a)',transform=ax[0].transAxes, fontweight="bold")
ax[1].text(0.03, 0.90, 'b)',transform=ax[1].transAxes, fontweight="bold")
ax[2].text(0.03, 0.90, 'c)',transform=ax[2].transAxes, fontweight="bold")
ax[3].text(0.03, 0.90, 'd)',transform=ax[3].transAxes, fontweight="bold")
ax[4].text(0.03, 0.90, 'e)',transform=ax[4].transAxes, fontweight="bold")
ax[5].text(0.03, 0.90, 'f)',transform=ax[5].transAxes, fontweight="bold")
ax[6].text(0.03, 0.90, 'g)',transform=ax[6].transAxes, fontweight="bold")
ax[7].text(0.03, 0.90, 'h)',transform=ax[7].transAxes, fontweight="bold")
ax[8].text(0.03, 0.90, 'i)',transform=ax[8].transAxes, fontweight="bold")
ax[9].text(0.03, 0.90, 'j)',transform=ax[9].transAxes, fontweight="bold")
ax[10].text(0.03, 0.90, 'k)',transform=ax[10].transAxes, fontweight="bold")
ax[11].text(0.03, 0.90, 'l)',transform=ax[11].transAxes, fontweight="bold")

plt.tight_layout(rect=[0.04, 0.025, 1, 0.96])
plt.savefig('/Users/jcaillet/Documents/Elmer/CMIP6/FIGURES/SLE_total_modif.pdf', dpi=300)